do pliku `build.gradle(Module)`

```kotlin
dependencies {

    implementation("androidx.lifecycle:lifecycle-viewmodel-compose:2.9.2")

    ...
}
```

```kotlin
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        enableEdgeToEdge()
        setContent {
            BlockingOperationsWithContextExampleTheme {
                HeavyCalculationScreen()
            }
        }
    }
}

data class UiState(
    val isLoading: Boolean = false,
    val result: String = "Gotowy do pracy."
)

class CalculationViewModel : ViewModel() {
    private val _uiState = MutableStateFlow(UiState())
    val uiState: StateFlow<UiState> = _uiState.asStateFlow()

    fun runBlockingCalculation() {
        viewModelScope.launch { // Startuje na Dispatchers.Main
            _uiState.value = UiState(
                isLoading = true,
                result = "Obliczenia blokujące START...")
            val calculationResult = performHeavyCalculation()
            _uiState.value = UiState(
                isLoading = false,
                result = "Blokujące ZAKOŃCZONE: $calculationResult")
        }
    }

    fun runCorrectCalculation() {
        viewModelScope.launch { // Startuje na Dispatchers.Main
            _uiState.value = UiState(
                isLoading = true,
                result = "Obliczenia w tle START...")
            val calculationResult = withContext(Dispatchers.Default) {
                performHeavyCalculation()
            }
            _uiState.value = UiState(
                isLoading = false,
                result = "Poprawne ZAKOŃCZONE: $calculationResult")
        }
    }

    private fun performHeavyCalculation(): Double {
        var tempResult = 0.0
        for (i in 0..40_000_000) {
            tempResult += sqrt(i.toDouble())
        }
        return tempResult
    }
}

// =================================================================================
// --- UI (Widok) ---
// =================================================================================

@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun HeavyCalculationScreen(viewModel: CalculationViewModel = viewModel()) {
    val uiState by viewModel.uiState.collectAsStateWithLifecycle()

    Scaffold(topBar = { TopAppBar(title = { Text("Demo withContext") }) }) { padding ->
        Column(
            modifier = Modifier
                .fillMaxSize()
                .padding(padding)
                .padding(16.dp),
            horizontalAlignment = Alignment.CenterHorizontally
        ) {
            Text(
                "Test Responsywności UI",
                style = MaterialTheme.typography.headlineSmall,
                textAlign = TextAlign.Center
            )
            Spacer(Modifier.height(16.dp))

            var sliderPosition by remember { mutableFloatStateOf(0f) }
            Slider(
                value = sliderPosition,
                onValueChange = { sliderPosition = it }
            )
            Text("Przesuń, aby sprawdzić, czy UI odpowiada", style = MaterialTheme.typography.labelSmall)
            Spacer(Modifier.height(32.dp))

            Text(
                text = uiState.result,
                style = MaterialTheme.typography.bodyLarge,
                textAlign = TextAlign.Center,
                modifier = Modifier.height(60.dp)
            )

            // Wskaźnik ładowania dla operacji w tle
            if (uiState.isLoading) {
                CircularProgressIndicator(modifier = Modifier.padding(vertical = 16.dp))
            } else {
                Spacer(Modifier.height(16.dp + 48.dp)) // Placeholder, by layout nie skakał
            }

            // Przyciski uruchamiające operacje
            Button(
                onClick = { viewModel.runBlockingCalculation() },
                enabled = !uiState.isLoading,
                modifier = Modifier.fillMaxWidth()
            ) {
                Text("❌ Uruchom Blokująco (BŁĄD)")
            }
            Spacer(Modifier.height(16.dp))
            Button(
                onClick = { viewModel.runCorrectCalculation() },
                enabled = !uiState.isLoading,
                modifier = Modifier.fillMaxWidth()
            ) {
                Text("✅ Uruchom Poprawnie (z withContext)")
            }
        }
    }
}
```